# GROUP 12 Coursework 3

In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import re
# from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv('Digital_Music_new.csv')

## Q1: Binary label
From the rating column, compute a binary label. I will use a cutoff threshold $C$. Ratings greater or equal to $C$ will be assigned label 1, ratings less than $C$ will be assigned label 0.

In [3]:
C = 4.0

In [4]:
df['Label'] = (df.Rating >= C).astype(np.int32)

In [5]:
df = df.dropna(subset=['Review'])  # Drop empty Review
df

,Unnamed: 0,Rating,Review,Summary,Time,Label
0,0,5.0,"It's hard to believe ""Memory of Trees"" came ou...",Enya's last great album,"09 12, 2006",1
1,1,5.0,"A clasically-styled and introverted album, Mem...",Enya at her most elegant,"06 3, 2001",1
2,2,5.0,I never thought Enya would reach the sublime h...,The best so far,"07 14, 2003",1
3,3,5.0,This is the third review of an irish album I w...,Ireland produces good music.,"05 3, 2000",1
4,4,4.0,"Enya, despite being a successful recording art...",4.5; music to dream to,"01 17, 2008",1
...,...,...,...,...,...,...
64701,64701,4.0,I like the reggae sound a lot in this song. I ...,Cool song,"06 24, 2014",1
64702,64702,5.0,I first heard this on Sirius and had to have i...,Great Song,"07 9, 2014",1
64703,64703,5.0,"I absolutely love this song, it downloaded fin...",Five Stars,"07 13, 2014",1
64704,64704,3.0,"Reggae, island beats aren't really my cup of t...",Well-crafted song,"07 9, 2014",0


Preprocessing

In [6]:
sw = stopwords.words('english')
stemmer = PorterStemmer()
def preprocess(text):
    #text is a document of a corpus
    #lowercase of text
    #remove all the irrelevant numbers and punctuation
    words = word_tokenize(re.sub(r'[^a-z]+', ' ', text.lower()))
    #remove the meaningless stopping words
    words = [t for t in words if t not in sw]
    #stemming transformation
    words = [stemmer.stem(t) for t in words]
    return words

In [7]:
# Takes time
df['Tokens'] = df.Review.apply(preprocess)

In [8]:
# import pickle
# with open('clean_data.pkl', 'wb') as f:
#     pickle.dump(df, f)

### Direct loading of cleaned data
To avoid long awaiting time start from here

In [2]:
# import pickle
# with open('clean_data.pkl', 'rb') as f:
#     df = pickle.load(f)

## Q2: Estimate a LASSO model
Predict the rating using text features. Tune regularization hyper-parameter by CV. How many terms got selected? Which five have the largest coefficients? which five have the smallest coefficients?

In [9]:
vectorizer = CountVectorizer(min_df=0.01)
dfm = vectorizer.fit_transform(list(df.Tokens.apply(' '.join)))
X_review_text_feature = dfm.toarray()
print("# of features", X_review_text_feature.shape[1])

# of features 1335


In [10]:
y = df.Label.values

In [11]:
# Split Train and test data
X_train, X_test, y_train, y_test = train_test_split(X_review_text_feature, y, test_size=0.1,
                                                   random_state=0)  # Ensure consistency

In [12]:
# Save train test data for R
pd.DataFrame(data=X_train, columns=vectorizer.get_feature_names()).to_csv('X_train.csv', index=False)
pd.DataFrame(data=X_test, columns=vectorizer.get_feature_names()).to_csv('X_test.csv', index=False)
pd.DataFrame(data=y_train, columns=['label']).to_csv('y_train.csv', index=False)
pd.DataFrame(data=y_test, columns=['label']).to_csv('y_test.csv', index=False)

In [27]:
# CV a LASSO
lasso_logistic = LogisticRegressionCV(Cs=np.arange(0.001, 2, 0.1),
                                      cv=5,
                                      penalty='l1',  # LASSO
                                      n_jobs=-1,  # Use all processors
                                      verbose=1,  # Report convergence
                                      solver='saga',  # Suppose L1 penalty
                                      tol=0.1, # Tolerance cannot be too low, take too much time,
                                      random_state=0, # Ensure reproductibility
                                     )


In [28]:
lasso_logistic.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 5 epochs took 6 seconds
convergence after 5 epochs took 6 seconds
convergence after 6 epochs took 7 seconds
convergence after 6 epochs took 7 seconds
convergence after 7 epochs took 12 seconds
convergence after 7 epochs took 13 seconds
convergence after 7 epochs took 13 seconds
convergence after 7 epochs took 13 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 3 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 4 seconds
convergence after 2 epochs took 3 seconds
convergence after 2 epochs took 3 seconds
convergence after 2 epochs took 3 seconds
convergence after 2 epochs took 3 seconds
convergence after 2 epochs took 3 seconds
convergence after 2 epochs too

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.2min finished


LogisticRegressionCV(Cs=array([1.000e-03, 1.010e-01, 2.010e-01, 3.010e-01, 4.010e-01, 5.010e-01,
       6.010e-01, 7.010e-01, 8.010e-01, 9.010e-01, 1.001e+00, 1.101e+00,
       1.201e+00, 1.301e+00, 1.401e+00, 1.501e+00, 1.601e+00, 1.701e+00,
       1.801e+00, 1.901e+00]),
                     class_weight=None, cv=5, dual=False, fit_intercept=True,
                     intercept_scaling=1.0, l1_ratios=None, max_iter=100,
                     multi_class='auto', n_jobs=-1, penalty='l1',
                     random_state=0, refit=True, scoring=None, solver='saga',
                     tol=0.1, verbose=1)

In [29]:
# The CV-ed optimal model paramters
lasso_logistic.C_  # The best C

array([1.901])

For the 10% sample held out as test data, the best model selected by CV predicts 86% out of them correctly.

In [30]:
# Report non-zero coefs
# Zip features and coefs
feature_coef = pd.DataFrame(data={'feature': vectorizer.get_feature_names(),
                                 'coef': lasso_logistic.coef_[0]})
feature_coef['abs_coef'] = np.abs(feature_coef.coef)
feature_coef = feature_coef.sort_values(by='abs_coef', ascending=False)

In [31]:
print('Total features', feature_coef.shape[0])
# Non zero coefs
print('Prominent features' , feature_coef[feature_coef.abs_coef >= 0.01].shape[0])

Total features 1335
Prominent features 1192


Out of 1335 terms in the feature space, 1183 terms have a coefficient greater than 0.01.  
The sparsity is not strong enforced due to CV-optimized paramter `C=1.901` (a bit large, i.e. regularization is a bit weak). But as we will see later, the logistic regression did a great job in identifying terms.

In [32]:
feature_coef

,feature,coef,abs_coef
1317,worst,-0.952711,0.952711
124,bore,-0.871155,0.871155
270,decent,-0.725420,0.725420
468,garbag,-0.715671,0.715671
647,lack,-0.680506,0.680506
...,...,...,...
1319,worthi,-0.000119,0.000119
1073,somehow,0.000109,0.000109
797,non,0.000078,0.000078
1112,statu,0.000075,0.000075


In [33]:
feature_coef.head(10)

,feature,coef,abs_coef
1317,worst,-0.952711,0.952711
124,bore,-0.871155,0.871155
270,decent,-0.725420,0.725420
468,garbag,-0.715671,0.715671
647,lack,-0.680506,0.680506
558,horribl,-0.598200,0.598200
1181,terribl,-0.585822,0.585822
1244,unfortun,-0.573425,0.573425
730,mediocr,-0.570304,0.570304
410,favorit,0.561093,0.561093


- The 5 most relevant terms to predict the rating are
    - worst
    - bore
    - decent
    - garbag
    - lack
- The 5 least relevant terms to predict the rating are
    - worthi
    - somehow
    - non
    - statu
    - nice
    
As we have expect, the negative ('pejorative') terms in top 5 are unanimously associated with a negative coefficient. That is, the more frequent the word appears in a review, the more likely it will receive label 0 (rating <= 3). If we expand to the top 10 terms, we see that complimentary terms like "favorit" and "awesom" receive positive coefficients, on the opposite.

## Q3 Estimate a Naive Bayes model
- Devise a metric that identifies the term most associated with each class label.
- How do these terms compare to those you identified in the previous question?

In [20]:
naive_bayes = MultinomialNB(alpha=1.0)

In [21]:
naive_bayes.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

We can extract the empirical log probability of features given a class, $P(x_i|y)$ by `feature_log_prob_` attribute. We postulate that, for a term $v_i$, if $P(v_i|y_i=0)$ is very different from $P(v_i|y_i=1)$, then $v_i$ is likely to be a discriminating term across the classes. Hence, we compute

$|P(v_i|y_i=0) - P(v_i|y_i=1)|$ for all $v_i\in V$

In [22]:
feature_prob_cls_0, feature_prob_cls_1 = np.exp(naive_bayes.feature_log_prob_)

In [23]:
feature_prob = pd.DataFrame(data={'feature': vectorizer.get_feature_names(),
                                  'prob_cls_0': feature_prob_cls_0,
                                  'prob_cls_1': feature_prob_cls_1})
feature_prob['abs_prob_diff'] = np.abs(feature_prob['prob_cls_0'] - feature_prob['prob_cls_1'])

In [24]:
feature_prob['Belong_To_Class'] = (feature_prob.prob_cls_0 > feature_prob.prob_cls_1).map({True: 0, False:1})
feature_prob.sort_values('abs_prob_diff', ascending=False).head(10)

,feature,prob_cls_0,prob_cls_1,abs_prob_diff,Belong_To_Class
675,like,0.017297,0.013247,0.004050,0
921,quot,0.018090,0.021805,0.003715,1
502,great,0.004938,0.008425,0.003487,1
695,love,0.005198,0.008024,0.002826,1
492,good,0.010347,0.007762,0.002586,0
818,one,0.011837,0.014188,0.002351,1
80,bad,0.003582,0.001522,0.002061,0
104,best,0.004976,0.006791,0.001815,1
105,better,0.004705,0.002900,0.001805,0
777,music,0.008701,0.010447,0.001745,1


With the metric we defined above, we identified the top 10 discriminating term and showed there association with a certain class. It is apparent that
- The terms are distinct from what we identified in Q2 with lasso logistic regression
- Although the 'discriminating' terms are pretty convince, as they are mostly very emotional words
- The classification is not very accurate. We have used a very simple criteria: the class with higher probability for the term is assigned. The drawback is that sometimes bi-grams like "not good", "don't love" appear in the complimentary class (label 1), and we brutally picked it up with "good" and "love"

This issue is associated with the fundamental independence assumption of Naive Bayes Classifier. Simply treating words as independent objects will ignore any negation or context.

## Q4 Estimate Multinomial Inverse Regression model
This part is implemented in R with `textir` package. We fitted a logistic regression as the forward regression.

The terms with heaviest loadings (sorted by absolute value of coefs) are:
- garbag
- wack
- horribl
- worst
- aw
- terribl
- wors
- mediocr
- suck
- timeless

They are all *strong* emotional wordings. The first nine (disapproving) are loaded with negative coefficients, while the last one 'timeless' is loaded with positive coefficient.

The terms with lightest loadings are:
- week
- went
... I will ignore the rest of them because they are all zeroed out.

The in-sample accuracy score is **0.8307**, quite close to the in-sample goodness-of-fit of LASSO regression **0.8597**.

## Q5 Out-of-sample CV comparison of LASSO, NB and MNIR

In [34]:
print('LASSO')
print('In-sample', accuracy_score(y_train, lasso_logistic.predict(X_train)))
print('Out-of-sample', accuracy_score(y_test, lasso_logistic.predict(X_test)))

LASSO
In-sample 0.860648418449703
Out-of-sample 0.8573636223149436


In [26]:
print('NB')
print('In-sample', accuracy_score(y_train, naive_bayes.predict(X_train)))
print('Out-of-sample', accuracy_score(y_test, naive_bayes.predict(X_test)))

NB
In-sample 0.7949994848370368
Out-of-sample 0.7853500231803431


The *in-sample* prediction accuracy (ranked) of
- LASSO is **0.860648418449703**
- MNIR is **0.830700278188**
- NB is **0.7949994848370368**

The *out-of-sample* prediction accuracy (ranked) of
- LASSO is **0.8573636223149436**
- NB is **0.7853500231803431**
- MNIR is **0.751090428272143**

Findings:
- Unanimously, the out-of-sample accuracy is lower than in-sample.
- The LASSO model outperforms the other two both in- and out-of sample. Though MNIR does better in-sample, Naive Bayes Classifier does better out-of-sample.